In [2]:
!pip install pysr --quiet

import os
import pandas as pd
import numpy as np
from pysr import PySRRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import itertools
import warnings
import random

# --- CONFIG ---
CSV_PATH = "/kaggle/input/dataset-for-symbolic-regression/data_augmented.csv"
TARGET = "tg"
BASE_FEATURES = [
    'mn','mw','dens','temperature','dielectric_constant',
    'molar_refractivity','radius_of_gyration_square','branching',
    'solvent_mol_wt','solvent_mol_mr','solvent_tpsa',
    'solvent_h_donors','solvent_h_acceptors'
]

# --- FIX RANDOM SEEDS ---
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'

# --- LOAD & CLEAN DATA ---
df = pd.read_csv(CSV_PATH)
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=BASE_FEATURES + [TARGET, "solvent_name"])

# Top 15 solvents
top_solvents = df["solvent_name"].value_counts().head(15).index.tolist()
df_top = df[df["solvent_name"].isin(top_solvents)].copy()
print("Top 15 solvents selected:", top_solvents)

# Features and target
X_df = df_top[BASE_FEATURES].copy()

# --- 🔹 ADD NEW DERIVED FEATURES ---
X_df["polar_density_ratio"] = X_df["dielectric_constant"] / X_df["dens"]
X_df["molecular_weight_ratio"] = X_df["mw"] / X_df["mn"]

# Replace any potential infinities from division
X_df.replace([np.inf, -np.inf], np.nan, inplace=True)
X_df.dropna(axis=0, inplace=True)

y = df_top.loc[X_df.index, TARGET].astype(float).values

# Clip extremes
X_df = X_df.clip(lower=X_df.quantile(0.01), upper=X_df.quantile(0.99), axis=1)

# Limited interaction features (first 20 pairwise products)
interaction_pairs = list(itertools.combinations(X_df.columns.tolist(), 2))[:20]
for a, b in interaction_pairs:
    X_df[f"{a}__x__{b}"] = X_df[a] * X_df[b]

FEATURES = X_df.columns.tolist()
print("Final feature count (including interactions):", len(FEATURES))

# Standardize
scaler = StandardScaler()
X = scaler.fit_transform(X_df.values)
print("Data shape (rows, cols):", X.shape)

# --- ALGEBRAIC-ONLY PySR MODEL (FULLY DETERMINISTIC) ---
model = PySRRegressor(
    niterations=400,
    population_size=50,
    ncycles_per_iteration=40,
    procs=1,
    batching=True,
    turbo=True,
    maxsize=25,
    model_selection="best",
    elementwise_loss="loss(x, y) = (x - y)^2",
    binary_operators=["+", "-", "*", "/"],
    unary_operators=[],
    constraints={"nvars": (3, None)},
    random_state=42,
    progress=True,
    verbosity=0,
)

warnings.filterwarnings("ignore")

# --- FIT ---
print("\n--- Running PySR (Algebraic-only model with 2 extra features) ---\n")
model.fit(X.astype(np.float32), y.astype(np.float32), variable_names=FEATURES)

# --- HALL OF FAME ---
hof_df = model.get_hof()
hof_csv_path = "hall_of_fame_algebraic_enhanced.csv"
hof_df.to_csv(hof_csv_path, index=False)
print("Hall of fame saved to:", hof_csv_path)

# --- SELECT BEST EQUATION (>=3 base features) ---
def count_base_features_in_eq(eq_str, base_features):
    return sum(1 for f in base_features if f in str(eq_str))

hof_df["n_base_features_in_eq"] = hof_df["equation"].apply(lambda s: count_base_features_in_eq(s, BASE_FEATURES))
candidates = hof_df[hof_df["n_base_features_in_eq"] >= 3].copy()

if candidates.shape[0] == 0:
    chosen = hof_df.sort_values(by=[c for c in hof_df.columns if "loss" in c.lower()], ascending=True).iloc[0]
else:
    loss_col = next((c for c in candidates.columns if "loss" in c.lower()), None)
    if loss_col is None:
        chosen = candidates.iloc[0]
    else:
        chosen = candidates.sort_values(by=loss_col, ascending=True).iloc[0]

selected_eq = chosen["equation"]
print("\n📘 Selected *Algebraic* Equation:")
print(selected_eq)

# --- PREDICTIONS & METRICS ---
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    y_pred = model.predict(X.astype(np.float32))

r2 = r2_score(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))
mae = mean_absolute_error(y, y_pred)

print("\n✅ Performance metrics on dataset:")
print(f"R²: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

# --- SAVE EQUATION ---
with open("best_tg_equation_algebraic_enhanced.txt", "w") as f:
    f.write(str(selected_eq))
print("\nSaved selected algebraic equation to 'best_tg_equation_algebraic_enhanced.txt' ✅")


Top 15 solvents selected: ['carbon dioxide', 'nitrogen', 'water', 'methane', 'benzene', 'n-pentane', 'toluene', 'methanol', 'n-butane', 'methylacetate', 'propylacetate', 'propane', 'acetone', 'methylethylketone', 'ethanol']
Final feature count (including interactions): 35
Data shape (rows, cols): (1044, 35)

--- Running PySR (Algebraic-only model with 2 extra features) ---



   Resolving package versions...
   Installed HostCPUFeatures ─── v0.1.17
   Installed SLEEFPirates ────── v0.6.43
   Installed SciMLPublic ─────── v1.0.0
   Installed Static ──────────── v1.3.1
   Installed CPUSummary ──────── v0.2.7
   Installed VectorizationBase ─ v0.21.72
   Installed LoopVectorization ─ v0.12.173
    Updating `~/.julia/environments/pyjuliapkg/Project.toml`
  [bdcacae8] + LoopVectorization v0.12.173
    Updating `~/.julia/environments/pyjuliapkg/Manifest.toml`
  [62783981] + BitTwiddlingConvenienceFunctions v0.1.6
  [2a0fbf3d] + CPUSummary v0.2.7
  [fb6a15b2] + CloseOpenIntervals v0.1.13
  [f70d9fcc] + CommonWorldInvalidations v1.0.0
  [adafc99b] + CpuId v0.3.1
  [3e5b6fbb] + HostCPUFeatures v0.1.17
  [615f187c] + IfElse v0.1.1
  [10f19ff3] + LayoutPointers v0.1.17
  [bdcacae8] + LoopVectorization v0.12.173
  [d125e4d3] + ManualMemory v0.1.8
  [6fe1bfb0] + OffsetArrays v1.17.0
  [1d0040c9] + PolyesterWeave v0.2.2
  [94e857df] + SIMDTypes v0.1.0
  [476501e8] + SLEEF

Hall of fame saved to: hall_of_fame_algebraic_enhanced.csv

📘 Selected *Algebraic* Equation:
((radius_of_gyration_square * -247.0023) - (mn__x__molecular_weight_ratio / -0.021465903)) + ((507.74213 - (mn__x__temperature * 53.731766)) - (148.91096 - ((-0.38501167 / mn__x__radius_of_gyration_square) + (molar_refractivity * (320.10513 - mn__x__radius_of_gyration_square)))))

✅ Performance metrics on dataset:
R²: 0.6757
RMSE: 74.9856
MAE: 57.5509

Saved selected algebraic equation to 'best_tg_equation_algebraic_enhanced.txt' ✅
